In [35]:
import sys
sys.path.append('../')
import safetensors
from build.lib.lora_diffusion import parse_safeloras
ckpt = safetensors.safe_open('../exps/output_dog_0.01/step_900.safetensors', framework="pt", device="cpu")
lora = parse_safeloras(ckpt)

In [17]:
from diffusers import StableDiffusionPipeline, EulerAncestralDiscreteScheduler
import torch
import os
model_id = "runwayml/stable-diffusion-v1-5"
device = torch.device("cuda:6")
pipe = StableDiffusionPipeline.from_pretrained(model_id, torch_dtype=torch.float16).to(
    "cuda:6"
)
pipe.scheduler = EulerAncestralDiscreteScheduler.from_config(pipe.scheduler.config)


Fetching 15 files: 100%|██████████| 15/15 [00:00<00:00, 49152.00it/s]


In [160]:
from lora_diffusion import parse_safeloras
c_reg = pipe._encode_prompt(['photo of a dog'],device,1,False)
ckpt = safetensors.safe_open('../exps/output_dog_0.1/step_900.safetensors', framework="pt", device="cpu")
lora = parse_safeloras(ckpt)  
up_weight_self = lora['unet'][0][10].to(device)
down_weight_self = lora['unet'][0][11].to(device)
up_weight = lora['unet'][0][12].to(device)
down_weight = lora['unet'][0][13].to(device)
lora_project_self = down_weight_self.transpose(0,1) @ up_weight_self.transpose(0,1)
lora_project = down_weight.transpose(0,1) @ up_weight.transpose(0,1)
lora_reside = c_reg @ lora_project
print(torch.norm(lora_project),torch.norm(lora_project_self))
torch.norm(lora_reside)

tensor(4.4258, device='cuda:6', dtype=torch.float16, grad_fn=<CopyBackwards>) tensor(2.2305, device='cuda:6', dtype=torch.float16, grad_fn=<CopyBackwards>)


tensor(24.0156, device='cuda:6', dtype=torch.float16, grad_fn=<CopyBackwards>)

In [155]:
from lora_diffusion.lora import _find_modules,DEFAULT_TARGET_REPLACE
cnt = 0
for target, name, module in _find_modules(pipe.unet, DEFAULT_TARGET_REPLACE):
    if module.in_features == 768:
        print(cnt, name)
    cnt += 1 

6 to_k
7 to_v
15 to_k
16 to_v
24 to_k
25 to_v
33 to_k
34 to_v
42 to_k
43 to_v
51 to_k
52 to_v
60 to_k
61 to_v
69 to_k
70 to_v
78 to_k
79 to_v
87 to_k
88 to_v
96 to_k
97 to_v
105 to_k
106 to_v
114 to_k
115 to_v
123 to_k
124 to_v
132 to_k
133 to_v
141 to_k
142 to_v
